## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Imbituba,BR,-28.2400,-48.6703,69.69,76,48,7.61,scattered clouds
1,1,Atuona,PF,-9.8000,-139.0333,79.34,69,18,14.18,few clouds
2,2,Rikitea,PF,-23.1203,-134.9692,76.10,81,69,12.80,broken clouds
3,3,Bluff,NZ,-46.6000,168.3333,49.80,70,78,4.74,broken clouds
4,4,Marawi,PH,7.9986,124.2928,74.64,83,98,4.54,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 83


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria using .loc
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.8000,-139.0333,79.34,69,18,14.18,few clouds
2,2,Rikitea,PF,-23.1203,-134.9692,76.10,81,69,12.80,broken clouds
4,4,Marawi,PH,7.9986,124.2928,74.64,83,98,4.54,light rain
5,5,Sao Geraldo Do Araguaia,BR,-6.4006,-48.5550,77.41,71,94,1.07,overcast clouds
7,7,Madimba,TZ,-10.4000,40.3333,70.36,71,81,15.30,broken clouds
11,11,Caravelas,BR,-17.7125,-39.2481,71.67,92,7,6.24,clear sky
13,13,Barcelona,ES,41.3888,2.1590,71.33,71,20,3.00,few clouds
25,25,Albany,US,42.6001,-73.9662,74.01,70,24,7.11,few clouds
26,26,Avarua,CK,-21.2078,-159.7750,75.25,69,40,8.05,scattered clouds
27,27,Puerto Ayora,EC,-0.7393,-90.3518,72.55,90,21,4.00,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pd.DataFrame(preferred_cities_df.dropna())
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
clean_df.count()

City_ID                216
City                   216
Country                216
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,79.34,few clouds,-9.8000,-139.0333,
2,Rikitea,PF,76.10,broken clouds,-23.1203,-134.9692,
4,Marawi,PH,74.64,light rain,7.9986,124.2928,
5,Sao Geraldo Do Araguaia,BR,77.41,overcast clouds,-6.4006,-48.5550,
7,Madimba,TZ,70.36,broken clouds,-10.4000,40.3333,
11,Caravelas,BR,71.67,clear sky,-17.7125,-39.2481,
13,Barcelona,ES,71.33,few clouds,41.3888,2.1590,
25,Albany,US,74.01,few clouds,42.6001,-73.9662,
26,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,
27,Puerto Ayora,EC,72.55,few clouds,-0.7393,-90.3518,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
#params = {
#    "radius": 5000,
#    "type": "lodging",
#    "key": g_key
#}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    hotels_found = 1
    # Add the latitude and longitude to location key for the params dictionary.
    params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location" : f"{lat},{lng}"
    }

    # Use the search term: "lodging" and our latitude and longitude.
    #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    google_nearby_base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #Make request for Google nearby search
    hotels = requests.get(google_nearby_base_url,params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
          
        #provides positive feedback that code is working by printing hotel added to df
        #print(hotel_df["Hotel Name"].count())
        #print(hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping.")
        #Provides positive feedback that iteration is working
        #print(f'{lat},{lng}')
#Prints number of hotel names added to hotel_df
print(f'{hotel_df["Hotel Name"].count()} Hotels Found')

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    

    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
216 Hotels Found


In [9]:
# 7. Drop the rows where there is no Hotel Name.
# None needed to be dropped

In [16]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [22]:
hotel_df.notnull().sum()

City                   216
Country                216
Max Temp               216
Current Description    216
Lat                    216
Lng                    216
Hotel Name             216
dtype: int64

In [20]:
hotel_df.count()

City                   216
Country                216
Max Temp               216
Current Description    216
Lat                    216
Lng                    216
Hotel Name             216
dtype: int64

In [23]:
clean_hotel_df = pd.DataFrame(hotel_df.dropna())

In [24]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,79.34,few clouds,-9.8000,-139.0333,Villa Enata
2,Rikitea,PF,76.10,broken clouds,-23.1203,-134.9692,Pension Maro'i
4,Marawi,PH,74.64,light rain,7.9986,124.2928,Derogongan Residence
5,Sao Geraldo Do Araguaia,BR,77.41,overcast clouds,-6.4006,-48.5550,SOLAR DO BARROSO
7,Madimba,TZ,70.36,broken clouds,-10.4000,40.3333,


In [38]:
# Null not caught by isnull or dropna (had to remove manually)
test_df = pd.DataFrame(hotel_df.dropna())
test_df[test_df['City']=="Madimba"].dropna()['Hotel Name']
test2_df = test_df[test_df['City']=="Madimba"].dropna()['Hotel Name']
test2_df

7    
Name: Hotel Name, dtype: object

In [25]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
clean_hotel_df = pd.read_csv('WeatherPy_vacation.csv')
clean_hotel_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Atuona,PF,79.34,few clouds,-9.8000,-139.0333,Villa Enata
1,2,Rikitea,PF,76.10,broken clouds,-23.1203,-134.9692,Pension Maro'i
2,4,Marawi,PH,74.64,light rain,7.9986,124.2928,Derogongan Residence
3,5,Sao Geraldo Do Araguaia,BR,77.41,overcast clouds,-6.4006,-48.5550,SOLAR DO BARROSO
4,11,Caravelas,BR,71.67,clear sky,-17.7125,-39.2481,Pousada dos Navegantes


In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
#create info box
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [58]:
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Villa Enata</dd>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Weather Description</dt><dd>few clouds</dd>\n<dt>Max Temp</dt><dd>79.34 °F</dd>\n</dl>\n',
 "\n<dl>\n<dt>Hotel Name</dt><dd>Pension Maro'i</dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Weather Description</dt><dd>broken clouds</dd>\n<dt>Max Temp</dt><dd>76.1 °F</dd>\n</dl>\n",
 '\n<dl>\n<dt>Hotel Name</dt><dd>Derogongan Residence</dd>\n<dt>City</dt><dd>Marawi</dd>\n<dt>Country</dt><dd>PH</dd>\n<dt>Weather Description</dt><dd>light rain</dd>\n<dt>Max Temp</dt><dd>74.64 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>SOLAR DO BARROSO</dd>\n<dt>City</dt><dd>Sao Geraldo Do Araguaia</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Weather Description</dt><dd>overcast clouds</dd>\n<dt>Max Temp</dt><dd>77.41 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Pousada dos Navegantes</dd>\n<dt>City</dt><dd>Caravelas</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Weather De

In [43]:
locations.head()

,Lat,Lng
0,-9.8000,-139.0333
1,-23.1203,-134.9692
2,7.9986,124.2928
3,-6.4006,-48.5550
4,-17.7125,-39.2481


In [59]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
#locations = hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))